In [1]:
import autogen
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

/opt/anaconda3/envs/agents/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config_list = [ 
    {
        "model": "llama3",
        "base_url": "http://localhost:11434/v1",
        "api_key": "ollama",
        "price": [0.01, 0.02],
    }
]

In [3]:
llm_config={
    "cache_seed":41,
    "temperature":0,
    "config_list":config_list,
    "timeout":120,
}

In [4]:
task = """Write a concise but engaging blogpost about Meta."""

In [5]:
writer = RetrieveAssistantAgent(
    name="Writer",
    llm_config={"config_list": config_list},
    system_message="""
    You are a professional writer, known for your insightful and engaging articles.
    You transform complex concepts into compelling narratives.
    You should improve the quality of the content based on the feedback from the user.
    """,
)

In [6]:
user_proxy = RetrieveUserProxyAgent(
    name="User",
    human_input_mode="NEVER",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    code_execution_config={
        "last_n_messages": 1,
        "work_dir": "my_code",
        "use_docker": False,
    }
)

2024-07-16 13:38:06,074 - autogen.agentchat.contrib.retrieve_user_proxy_agent - WARNING - docs_path is not provided in retrieve_config. Will raise ValueError if the collection `autogen-docs` doesn't exist. Set docs_path to None to suppress this warning.


In [8]:
critic = RetrieveAssistantAgent(
    name="Critic",
    llm_config={"config_list": config_list},
    system_message="""
    You are a critic, known for your thoroughness and commitment to standards.
    Your task is to scrutinize content for any harmful elements or regulatory violations, ensuring
    all materials align with required guidelines.
    For code
    """,
)

In [9]:
def reflection_message(recipient, messages, sender, config):
    print("Reflecting...")
    return f"Reflect and provide critique on the following writing. \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}"

In [10]:
user_proxy.register_nested_chats(
    [
        {
            "recipient": critic,
            "message": reflection_message,
            "summary_method": "last_msg",
            "max_turns": 1
         }
    ],
    trigger=writer
)

In [11]:
user_proxy.initiate_chat(recipient=writer, message=task, max_turns=2, summary_method="last_msg")

User (to Writer):

Write a concise but engaging blogpost about Meta.

--------------------------------------------------------------------------------
Writer (to User):

**The Rise of Meta: Revolutionizing Online Interactions**

In today's digital landscape, online interactions have become an integral part of our daily lives. From social media to virtual reality, the world is increasingly connected. Amidst this sea change, one company stands out for its innovative approach – Meta.

Founded in 2004 as Facebook, Meta has evolved into a behemoth of interconnectedness, with a portfolio that includes Instagram, WhatsApp, and more. Mark Zuckerberg's brainchild has grown from a college experiment to a global phenomenon, shaping the way we interact online.

**The Power of Social**

Facebook, the crown jewel of Meta's empire, revolutionized social media by giving people a platform to share their thoughts, connect with friends and family, and build communities around shared interests. Its influe

ChatResult(chat_id=None, chat_history=[{'content': 'Write a concise but engaging blogpost about Meta.', 'role': 'assistant'}, {'content': "**The Rise of Meta: Revolutionizing Online Interactions**\n\nIn today's digital landscape, online interactions have become an integral part of our daily lives. From social media to virtual reality, the world is increasingly connected. Amidst this sea change, one company stands out for its innovative approach – Meta.\n\nFounded in 2004 as Facebook, Meta has evolved into a behemoth of interconnectedness, with a portfolio that includes Instagram, WhatsApp, and more. Mark Zuckerberg's brainchild has grown from a college experiment to a global phenomenon, shaping the way we interact online.\n\n**The Power of Social**\n\nFacebook, the crown jewel of Meta's empire, revolutionized social media by giving people a platform to share their thoughts, connect with friends and family, and build communities around shared interests. Its influence is undeniable, with